In [1]:
import numpy as np
import pandas as pd
from os import walk
import os
#from google.colab import drive

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

#NLP stuff
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

# sklearn stuff
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kdmei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# read the speeches data from the csv (change the path so that it works for you)
df = pd.read_csv('raw_data.csv').set_index(['Year', 'ISO-alpha3 Code'])

# create a subset (this step can be skipped by putting a # in front of it)
df = df.loc[2005:]

In [86]:
# create a count vectorizer that 
vectorizer = TfidfVectorizer(stop_words = ['english'], token_pattern=r'\b[^\d\W\_]+\b', max_df = 0.95, min_df = 0.01)

# fit the speech data and transform it into a sparse matrix
X = vectorizer.fit_transform(df['Speech'].to_numpy())

In [87]:
# create a dataframe using the dense count matrix, index of the original speeches data and the features from the vectorizer
count_vect_df = pd.DataFrame(X.todense(), index=df.index, columns=vectorizer.get_feature_names())

In [88]:
# vectorizer.stop_words_
count_vect_df

ababa  abandon  abandoned  abandoning  abbas  abdulaziz  \
Year ISO-alpha3 Code                                                            
2005 AFG                0.0      0.0        0.0         0.0    0.0        0.0   
     AGO                0.0      0.0        0.0         0.0    0.0        0.0   
     ALB                0.0      0.0        0.0         0.0    0.0        0.0   
     AND                0.0      0.0        0.0         0.0    0.0        0.0   
     ARE                0.0      0.0        0.0         0.0    0.0        0.0   
...                     ...      ...        ...         ...    ...        ...   
2020 WSM                0.0      0.0        0.0         0.0    0.0        0.0   
     YEM                0.0      0.0        0.0         0.0    0.0        0.0   
     ZAF                0.0      0.0        0.0         0.0    0.0        0.0   
     ZMB                0.0      0.0        0.0         0.0    0.0        0.0   
     ZWE                0.0      0.0        0.0         0.0    0.0        0.0   

                      abdullah  abdussalam  abide  abiding  ...     youth  \
Year ISO-alpha3 Code                                        ...             
2005 AFG                   0.0         0.0    0.0      0.0  ...  0.000000   
     AGO                   0.0         0.0    0.0      0.0  ...  0.000000   
     ALB                   0.0         0.0    0.0      0.0  ...  0.000000   
     AND                   0.0         0.0    0.0      0.0  ...  0.000000   
     ARE                   0.0         0.0    0.0      0.0  ...  0.000000   
...                        ...         ...    ...      ...  ...       ...   
2020 WSM                   0.0         0.0    0.0      0.0  ...  0.018118   
     YEM                   0.0         0.0    0.0      0.0  ...  0.000000   
     ZAF                   0.0         0.0    0.0      0.0  ...  0.000000   
     ZMB                   0.0         0.0    0.0      0.0  ...  0.093160   
     ZWE                   0.0         0.0    0.0      0.0  ...  0.000000   

                      yugoslavia  zealand  zero  zimbabwe      zone  zones  \
Year ISO-alpha3 Code                                                         
2005 AFG                     0.0      0.0   0.0  0.000000  0.000000    0.0   
     AGO                     0.0      0.0   0.0  0.000000  0.000000    0.0   
     ALB                     0.0      0.0   0.0  0.000000  0.023810    0.0   
     AND                     0.0      0.0   0.0  0.000000  0.000000    0.0   
     ARE                     0.0      0.0   0.0  0.000000  0.033786    0.0   
...                          ...      ...   ...       ...       ...    ...   
2020 WSM                     0.0      0.0   0.0  0.000000  0.000000    0.0   
     YEM                     0.0      0.0   0.0  0.000000  0.000000    0.0   
     ZAF                     0.0      0.0   0.0  0.042009  0.000000    0.0   
     ZMB                     0.0      0.0   0.0  0.000000  0.000000    0.0   
     ZWE                     0.0      0.0   0.0  0.442099  0.000000    0.0   

                             à  état  être  
Year ISO-alpha3 Code                        
2005 AFG              0.000000   0.0   0.0  
     AGO              0.000000   0.0   0.0  
     ALB              0.039251   0.0   0.0  
     AND              0.000000   0.0   0.0  
     ARE              0.000000   0.0   0.0  
...                        ...   ...   ...  
2020 WSM              0.000000   0.0   0.0  
     YEM              0.000000   0.0   0.0  
     ZAF              0.000000   0.0   0.0  
     ZMB              0.000000   0.0   0.0  
     ZWE              0.000000   0.0   0.0  

[3086 rows x 6806 columns]

In [89]:
count_vect_df.to_csv('tfidf.csv')